In [ ]:
import gurobipy as gp
from gurobipy import GRB
from automata.fa.dfa import DFA
from pydot import Dot, Edge, Node
import itertools
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from time import time
import pandas as pd

def data_preprocess():

    # Read the CSV file
    df = pd.read_csv(r'C:\Users\bchan\Downloads\Dodgers\univariate\Dodgers\101-freeway-traffic.test.csv')

    # Convert the 'timestamp' column to datetime format
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Add a new column for day of the week (0 = Monday, 6 = Sunday)
    df['day_of_week'] = df['timestamp'].dt.dayofweek

    # Filter only rows corresponding to each days
    mon = df[df['day_of_week'] == 0]
    tue = df[df['day_of_week'] == 1]
    wed = df[df['day_of_week'] == 2]
    thu = df[df['day_of_week'] == 3]
    fri = df[df['day_of_week'] == 4]
    sat = df[df['day_of_week'] == 5]
    sun = df[df['day_of_week'] == 6]

    #print(mon)
    #print(sum(sun['count'].values[0:48]))
    sun_list, mon_list, tue_list, wed_list, thu_list, fri_list, sat_list =[], [], [], [], [], [], []
    sun_label, mon_label, tue_label, wed_label, thu_label, fri_label, sat_label=[], [], [], [], [], [], []

    #288 data points corresponds to 1day(24hrs). 1hrs-12obs. 6hrs-72 obs
    for i in range(0, len(sun['count'].values), 288):
        #print(f'i:{i}')
        if -1 not in sun['count'].values[i:i+288] and 1 not in sun['is_anomaly'].values[i:i+288]:
            sun_list.append([sum(sun['count'].values[j:j+72]) for j in range(i, i+288, 72)])
            sun_label.append([sum(sun['is_anomaly'].values[j:j+72]) for j in range(i, i+288, 72)])
        if -1 not in mon['count'].values[i:i+288]:
            mon_list.append([sum(mon['count'].values[j:j+72]) for j in range(i, i+288, 72)])
            mon_label.append([sum(mon['is_anomaly'].values[j:j+72]) for j in range(i, i+288, 72)])
        if -1 not in tue['count'].values[i:i+288]:
            tue_list.append([sum(tue['count'].values[j:j+72]) for j in range(i, i+288, 72)])
            tue_label.append([sum(tue['is_anomaly'].values[j:j+72]) for j in range(i, i+288, 72)])
        if -1 not in wed['count'].values[i:i+288]:
            wed_list.append([sum(wed['count'].values[j:j+72]) for j in range(i, i+288, 72)])
            wed_label.append([sum(wed['is_anomaly'].values[j:j+72]) for j in range(i, i+288, 72)])
        if -1 not in thu['count'].values[i:i+288]:
            thu_list.append([sum(thu['count'].values[j:j+72]) for j in range(i, i+288, 72)])
            thu_label.append([sum(thu['is_anomaly'].values[j:j+72]) for j in range(i, i+288, 72)])
        if -1 not in fri['count'].values[i:i+288] and 1 not in fri['is_anomaly'].values[i:i+288]:
            fri_list.append([sum(fri['count'].values[j:j+72]) for j in range(i, i+288, 72)])
            fri_label.append([sum(fri['is_anomaly'].values[j:j+72]) for j in range(i, i+288, 72)])
        if -1 not in sat['count'].values[i:i+288]:
            sat_list.append([sum(sat['count'].values[j:j+72]) for j in range(i, i+288, 72)])
            sat_label.append([sum(sat['is_anomaly'].values[j:j+72]) for j in range(i, i+288, 72)])
    
    rounded_satlist=[]
    for i in sat_list:
        rounded_satlist.append([round(j/100) for j in i])

    #Create labels
    G=[]
    for i in sat_label:
        if any(i):
            G.append(1)
        else:
            G.append(0)
        
    #Adding normal sequence to increase the number of normal sequence
    #for saturday
    repeat=[[3, 14, 20, 16], [3, 11, 17, 17], [4, 16, 19, 16],[3, 13, 18, 16], [4, 11, 18, 15], [4, 14, 20, 17], [4, 13, 18, 15],[4, 13, 17, 13]]
    i=0
    while i < 11:    
        for r in repeat:
            rounded_satlist.append(r)
            G.append(0)
        i+=1

    #manual labelling of sequence considering very low traffic also as anamaly
    G[0]=1
    G[13]=1

    FL_dist=rounded_satlist
    #G_train=G

    FL_dist_test=[]
    G_test=[]
    for i in range(7):
        FL_dist_test.append(rounded_satlist[i])
        G_test.append(G[i])
    for i in range(len(rounded_satlist)-15, len(rounded_satlist)):
        FL_dist_test.append(rounded_satlist[i])
        G_test.append(G[i])

    Lst=[]
    Lst_test=[]
    for i in FL_dist:
        Lst.append(str(i)[1:-1].replace(' ',''))
    for i in FL_dist_test:
        Lst_test.append(str(i)[1:-1].replace(' ',''))

    FL = [l.split(',') for l in Lst]
    FL_test = [l.split(',') for l in Lst_test]

    uniq_list=set(Lst)
    alphabet = set(item for string in uniq_list for item in string.split(",") if item != ',')

    Pref_S = set()
    for string in uniq_list:
        prefixes = string.split(',')
        for i in range(1, len(prefixes) + 1):
            prefix = ','.join(prefixes[:i])
            Pref_S.add(prefix)
    Pref_S.add('')
    
    return alphabet, Pref_S, Lst, FL, FL_test, G_test


def train(n, lower, upper):
    alphabet, Pref_S, Lst, FL, FL_test, G_test = data_preprocess()
    states = {str(f'q{i}') for i in range(n)}
    start_state = 'q0'

    env=gp.Env(empty=True)
    env.setParam('OutputFlag', 0)
    env.start()

    t0 = time()
    # Creating a new model
    mdbtc = gp.Model("DFA_DBTC", env=env)

    #DECISION VARIABLES
    delta = mdbtc.addVars(states, alphabet, states, vtype=gp.GRB.BINARY, name='delta')
    x = mdbtc.addVars(Pref_S, states, vtype=gp.GRB.BINARY, name='x')
    f = mdbtc.addVars(states, vtype=gp.GRB.BINARY, name='f')
    alpha = mdbtc.addVars(len(Lst), states, vtype=gp.GRB.BINARY, name= 'alpha')
    LB = mdbtc.addVar(lb=lower,ub=lower,vtype=gp.GRB.CONTINUOUS, name='LB')
    UB = mdbtc.addVar(lb=upper, ub=upper,vtype=gp.GRB.CONTINUOUS, name='UB')

    #OBJECTIVE
    #Self-loop: sum(delta[state0,symbol,state1] for state0 in states for symbol in alphabet for state1 in states if state0 != state1)
    mdbtc.setObjective(1, gp.GRB.MINIMIZE)

    #AUTOMATA CONSTRAINTS
    #Constraint1
    for state0 in states:
        for symbol in alphabet:
            mdbtc.addConstr(sum(delta[state0,symbol,state1] for state1 in states)==1, name=f'delta[{state0},{symbol}]')
    
    #Constraint2
    for word in Pref_S:
        mdbtc.addConstr(sum(x[word,state1] for state1 in states)==1, name=f'x[{word}]')


    #Constraint3
    mdbtc.addConstr(x['',start_state]==1, name='initial_state')

    #Constraint4
    for state0, word, symbol, state1 in itertools.product(states, Pref_S, alphabet, states):
        if (word + ',' + symbol) in Pref_S:
            mdbtc.addConstr(x[word,state0] + delta[state0, symbol, state1] -1 <= x[word + ',' + symbol, state1], name=f'transition[{state0},{word},{symbol},{state1}]')

        if word == '' and symbol in Pref_S:
            mdbtc.addConstr(x[word, state0] + delta[state0, symbol, state1] - 1 <= x[symbol, state1], name=f'transition[{state0},{word},{symbol},{state1}]')


    #BOUND CONSTRAINTS
    for i, word in enumerate(Lst):
        for state1 in states:
            mdbtc.addConstr(alpha[i, state1] >= x[word,state1] + f[state1] -1, name=f'bound_1[{state1},{i}]')
            mdbtc.addConstr(alpha[i, state1] <= x[word,state1], name=f'bound_2[{state1},{i}]')
            mdbtc.addConstr(alpha[i, state1] <= f[state1], name=f'bound_3[{state1},{i}]')        
         
    mdbtc.addConstr(sum(alpha[i, state1] for i,word in enumerate(Lst) for state1 in states )/len(Lst) >= LB, name=f'lowerBound')
    mdbtc.addConstr(sum(alpha[i, state1] for i,word in enumerate(Lst) for state1 in states )/len(Lst) <= UB, name=f'upperBound')

    #Write the model
    mdbtc.write(rf'C:\Users\bchan\Desktop\TUD\Thesis\model_DB_{n}.lp')

    mdbtc.optimize()
    #print('Obj: %g' % mdbtc.ObjVal)

    t1 = time()
    print("Run time", (t1-t0), "seconds")

    if mdbtc.status == 1:
        status = 'LOADED'
        print(f'DFAmodel_{n}: {status}')
            
    elif mdbtc.status == 2:
        print(f'DFAmodel_{n} OPTIMAL')
        status='OPTIMAL'
        transitions = mdbtc.getAttr('X', delta)
        t_values = [(s1,a,s2) for s1 in states for s2 in states for a in alphabet if round(transitions[s1, a, s2],0) == 1]
        f_s = mdbtc.getAttr('X', f)
        #print(f_s)
        final_state = {s1 for s1 in states if round(f_s[s1],0) == 1}

        transition_dict = create_transition_dict(states, alphabet, t_values)
        
        dfa1 = DFA(states=states,input_symbols=alphabet, transitions= transition_dict, initial_state= start_state, final_states=final_state)
        #print(f'Final_state:{final_state}')
        accepted = 0
        rejected = 0
        for w in FL:
            if dfa1.accepts_input(w):
                #print(f'{w}:accepted')
                #print('accepted')
                accepted += 1             
            else:
                rejected += 1        
        print(f'Accepted:{accepted}')
        print(f'Rejected:{rejected}')

        create_diagram(rf'C:\Users\bchan\Desktop\TUD\Thesis\diagram_DB_{n}.png', states, start_state,final_state, transition_dict)
        return dfa1, FL_test, G_test 
    
    elif mdbtc.status == 3:
        status = 'INFEASIBLE'
        print(f'DFAmodel_{n}: {status}')
    else:
        print('status unknown, DEBUG!!')    


def create_transition_dict(states, alphabet, t_values):
    transition_dict = {}

    for state in states:
        transition_dict[state] = {}
        for symbol in alphabet:
            transition_dict[state][symbol] = None

    for trans in t_values:
        current_state, symbol, next_state = trans
        transition_dict[current_state][symbol] = next_state

    return transition_dict

def create_diagram(path, states, start_state, final_state, transition_dict):

    graph = Dot(graph_type='digraph', rankdir='LR')
    nodes = {}
    for state in states:
        if state == start_state:
            # color start state with green
            if state in final_state:
                initial_state_node = Node(
                    state,
                    style='filled',
                    peripheries=2,
                    fillcolor='#66cc33')
            else:
                initial_state_node = Node(
                    state, style='filled', fillcolor='#66cc33')
            nodes[state] = initial_state_node
            graph.add_node(initial_state_node)
        else:
            if state in final_state:
                state_node = Node(state, peripheries=2)
            else:
                state_node = Node(state)
            nodes[state] = state_node
            graph.add_node(state_node)
    # adding edges
    for from_state, lookup in transition_dict.items():
        for to_label, to_state in lookup.items():
            graph.add_edge(Edge(
                nodes[from_state],
                nodes[to_state],
                label=to_label
            ))
    if path:
        graph.write_png(path)
    return graph

def test(G_test, FL_test, dfa1):
    accepted = 0
    rejected = 0
    Predicted_labels=[]

    for w in FL_test:
        #print(w)
        if dfa1.accepts_input(w):
            Predicted_labels.append(1)
            #print(f'{w}:accepted')
            accepted += 1             
        else:
            #print(f'{w}:rejected')
            Predicted_labels.append(0)
            rejected += 1  
    
    print(f'Accepted in Testing:{accepted}')
    print(f'Rejected in Testing:{rejected}')
    #print(f'Predicted_labels:{Predicted_labels}')
    #print(f'True_labelssssss:{G_test}')
    
    accuracy = accuracy_score(G_test, Predicted_labels)
    print(f'Accuracy:{round(accuracy,2)}')
    f1 = f1_score(G_test, Predicted_labels, average='binary', pos_label=1)
    print(f'F1_score:{round(f1,2)}\n')


for n in range(2,3):
    lower=0.10
    upper=0.11
    dfa1, FL_test, G_test = train(n, lower, upper)
    test(G_test, FL_test, dfa1)